In [4]:
import dart_fss as dart
import pandas as pd
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
dart.set_api_key(os.getenv("DART_API_KEY"))

# 저장 경로
output_dir = "total_dart"
os.makedirs(output_dir, exist_ok=True)

# corp_code 목록 로드
corp_df = pd.read_csv("corp_code.csv", dtype={'corp_code': str})
corp_df['corp_code'] = corp_df['corp_code'].str.zfill(8)

# 기업별 다운로드 및 저장 함수
def extract_and_save_fs(corp_code):
    try:
        fs = dart.fs.extract(
            corp_code=corp_code,
            bgn_de='20150101',
            fs_tp=('bs', 'is', 'cis', 'cf'),
            report_tp='quarter',
            cumulative=False,
            skip_error=True,
            last_report_only=False
        )

        all_dfs = []
        for tp in ['bs', 'is', 'cis', 'cf']:
            try:
                df = fs[tp].copy()
                if df.empty:
                    continue
                df.reset_index(inplace=True)
                df.columns = [str(c) for c in df.columns]
                df.insert(0, 'corp_code', corp_code)
                df.insert(1, 'fs_tp', tp.upper())
                all_dfs.append(df)
            except Exception as e:
                print(f"{corp_code} - {tp} 오류:", e)

        if all_dfs:
            merged_df = pd.concat(all_dfs, ignore_index=True)
            account_col = merged_df.columns[2]
            fixed = ['corp_code', 'fs_tp', account_col]
            time_cols = sorted([col for col in merged_df.columns if col not in fixed])
            merged_df = merged_df[fixed + time_cols]
            merged_df.to_csv(f"{output_dir}/{corp_code}_2015_전체재무제표.csv", index=False, encoding='utf-8-sig')
            print(f"✅ 저장 완료: {corp_code}")
        else:
            print(f"⚠️ 데이터 없음: {corp_code}")
    except Exception as e:
        print(f"❌ 전체 실패: {corp_code}", e)

# 전체 처리
for corp_code in corp_df['corp_code']:
    extract_and_save_fs(str(corp_code))


TypeError: 'type' object is not subscriptable